## Initializing Spark Environment and Loading Data

In [1]:
import findspark
findspark.init('/home/shahayush954/spark-3.4.1-bin-hadoop3')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('trees').getOrCreate()

23/08/19 18:57:09 WARN Utils: Your hostname, ubuntu-22 resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/08/19 18:57:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/19 18:57:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/19 18:57:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
data = spark.read.csv('College.csv', header=True, inferSchema=True)

In [4]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



## Data Preparation and Transformation

In [5]:
from pyspark.ml.feature import VectorAssembler

In [6]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [7]:
assembler = VectorAssembler(inputCols=['Apps','Accept','Enroll',
                                       'Top10perc','Top25perc','F_Undergrad',
                                       'P_Undergrad','Outstate','Room_Board',
                                       'Books','Personal','PhD',
                                       'Terminal','S_F_Ratio','perc_alumni',
                                       'Expend','Grad_Rate'], outputCol='features')

In [8]:
feature_data = assembler.transform(data)

In [9]:
from pyspark.ml.feature import StringIndexer
stringIndexer = StringIndexer(inputCol='Private', outputCol='PrivateIndex')

In [10]:
feature_data = stringIndexer.fit(feature_data).transform(feature_data)

In [11]:
feature_data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)



In [12]:
final_data = feature_data.select(['features', 'PrivateIndex'])

In [13]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

## Decision Trees

In [14]:
from pyspark.ml.classification import DecisionTreeClassifier

In [16]:
dtc = DecisionTreeClassifier(labelCol='PrivateIndex', featuresCol='features')

In [17]:
dtc_model = dtc.fit(train_data)

In [18]:
dtc_predictions = dtc_model.transform(test_data)

## Random Forests

In [19]:
from pyspark.ml.classification import RandomForestClassifier

In [20]:
rfc = RandomForestClassifier(labelCol='PrivateIndex', featuresCol='features')

In [21]:
rfc_model = rfc.fit(train_data)

In [22]:
rfc_predictions = rfc_model.transform(test_data)

## Gradient Boost Trees

In [23]:
from pyspark.ml.classification import GBTClassifier

In [24]:
gbt = GBTClassifier(labelCol='PrivateIndex', featuresCol='features')

In [25]:
gbt_model = gbt.fit(train_data)

In [26]:
gbt_predictions = gbt_model.transform(test_data)

## Evaluate Performance

In [27]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [28]:
evaluator = BinaryClassificationEvaluator(labelCol='PrivateIndex')

In [31]:
print('DTC: ' + str(evaluator.evaluate(dtc_predictions)))

DTC: 0.8945705718569287


In [32]:
print('RFC: ' + str(evaluator.evaluate(rfc_predictions)))

RFC: 0.9823302634397086


In [33]:
print('GBT: ' + str(evaluator.evaluate(gbt_predictions)))

GBT: 0.9443135575069609


In [34]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [35]:
multi_evaluator = MulticlassClassificationEvaluator(labelCol='PrivateIndex', metricName='accuracy')

In [36]:
multi_evaluator.evaluate(rfc_predictions)

0.9452054794520548